# Bitcoin Key Pairs

In [1]:
# pip install ecdsa

## Manual Key Generation

In [2]:
import ecdsa

# Generate a private key
private_key = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
print("Private key:", private_key.to_string().hex())

# Generate a public key
public_key = private_key.get_verifying_key()
print("Public key: ", public_key.to_string().hex())

Private key: d3ccfb67695cd047f8e4cabd376131ee74a89b2e3d778d393467499e05280882
Public key:  291ecfe7510b332c277e91ac3e6997fd550c4f464738d90ca92f1a94cee8a00721c77e38add025ce3327bc837dd3d8335bea16c7a5aceb6d51fa8b83a3399e9b


While the public key is correct it is not in the proper format. 

## Bitcoin Address

Finding the bitcoin public key or bitcoin address from the private key is a bit more complicated.

The public key is a point on the elliptic curve, but it is not in the proper format. The proper format is a 65 byte array with the first byte being 0x04 and the next 32 bytes being the x coordinate and the last 32 bytes being the y coordinate. The x and y coordinates are 32 bytes each because they are 256 bit numbers. The first byte is 0x04 because it is an uncompressed public key. There are compressed public keys which are 33 bytes long, but we will not be using them.

In [3]:
import hashlib
import base58

# Generate a bitcoin address
# Step 1: Hash public key
sha256 = hashlib.sha256(public_key.to_string()).digest()

# Step 2: Perform RIPEMD-160 hashing on the result of SHA256
ripemd160 = hashlib.new('ripemd160')
ripemd160.update(sha256)
hashed_public_key = ripemd160.digest()

# Step 3: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)
hashed_public_key = b'\x00' + hashed_public_key

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(hashed_public_key).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]

# Step 6: Add the 4 checksum bytes from stage 5 at the end of extended RIPEMD-160 hash from stage 4. This is the 25-byte binary Bitcoin Address.
binary_bitcoin_address = hashed_public_key + address_checksum

# Step 7: Convert the result from a byte string into a base58 string using Base58Check encoding. This is the most commonly used Bitcoin Address format
bitcoin_address = base58.b58encode(binary_bitcoin_address)
print("Bitcoin address: ", bitcoin_address.decode('utf-8'))

Bitcoin address:  1EWCJMiduJVVbHDk9xMCLUBPW6AUc8Dmir


In [11]:
# Verify that the bitcoin address is correct
# Step 1: Decode Base58 Bitcoin Address
binary_bitcoin_address_check = base58.b58decode(bitcoin_address)

# Step 2: Extract Bitcoin Address from decoded Base58 Bitcoin Address
binary_bitcoin_address = binary_bitcoin_address_check[:-4]

# Step 3: Extract Bitcoin Address Checksum from decoded Base58 Bitcoin Address
binary_bitcoin_address_checksum = binary_bitcoin_address_check[-4:]

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(binary_bitcoin_address).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]

# Step 6: Verify that the address checksum from decoded Base58 Bitcoin Address is equal to the address checksum from the SHA256 hash
print("Bitcoin address is valid:", binary_bitcoin_address_checksum == address_checksum)

AttributeError: 'P2pkhAddress' object has no attribute 'decode'

# Libraries for bitcoin keys

### pybitcoin

No longer maintained and does not work with python 3.12

In [ ]:
# pip install pybitcoin

In [ ]:
# from pybitcoin import BitcoinPrivateKey

# # Generate a private key
# private_key = BitcoinPrivateKey()
# print(f"Private key: {private_key.to_hex()}")

# # Generate a public key
# public_key = private_key.public_key()
# print(f"Public key: {public_key.to_hex()}")

# # Generate a bitcoin address
# bitcoin_address = public_key.address()
# print(f"Bitcoin address: {bitcoin_address}")

### bitcoin-utils

In [ ]:
# pip install bitcoin-utils

In [ ]:
from bitcoinutils.keys import PrivateKey, PublicKey

# Generate a private key
private_key = PrivateKey()
print(f"Private key: {private_key.to_wif(compressed=True)}")

# get the public key
public_key: PublicKey = private_key.get_public_key()
print(f"Public key: {public_key.to_hex(compressed=True)}")

# get the bitcoin address
bitcoin_address = public_key.get_address()
print(f"Bitcoin address: {bitcoin_address.to_string()}")
print(f'Hash160: {bitcoin_address.to_hash160()}')



Private key: cRxpTezWUeah7BTvWirZfLUwgQiMhCnYvzXaEMwmddgSAt7jM1uU
Public key: 028214fbab7d7a4aa5edc74cfb4273c8efefa5c64291f40aacc8acaaa5377a99e9
Bitcoin address: moC7rDoqzgf3uQgqA9Swz6LHcUpQsVsst6
Hash160: 5431142b58ab8e537c4051d28f537da52fd792a7


Sign a message with a private key and verify the signature with the public key.

In [ ]:
message: str = "Hello"
signature = private_key.sign_message(message)
print(f"Signature: {signature}")
# print(f"Signature valid? {public_key.verify(message, signature)}")
print(f"Signature valid? {public_key.verify_message(public_key.get_address().to_string(), message, signature)}")

Signature: None


ValueError: Invalid address